In [1]:
!pip install langchain_groq langchain_core langchain_community

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is Jesus")
print(result.content)


Jesus, also known as Jesus of Nazareth or Jesus Christ, is a central figure in Christianity and one of the most influential people in human history. He is believed by Christians to be the Son of God, the Messiah, and the savior of humanity.

According to the New Testament of the Bible, Jesus was born in Bethlehem, in the Roman province of Judea, around 4 BC. He was born to a Jewish family, with Mary as his mother and Joseph as his adoptive father. Jesus' birth is often seen as a miraculous event, with the Bible describing it as a virgin birth, where Mary conceived Jesus through the Holy Spirit.

Jesus grew up in Nazareth, where he was raised by his parents and learned the trade of carpentry from his father. At the age of 30, Jesus began his public ministry, preaching and teaching about the kingdom of God. He traveled throughout the region, performing miracles, healing the sick, and teaching about God's love and forgiveness.

Jesus' teachings emphasized the importance of love, compassio

In [3]:
!pip install pypdf

In [4]:
!pip install chromadb

  Using cached chromadb-1.0.0-cp39-abi3-win_amd64.whl.metadata (7.0 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6-cp312-cp312-win_amd64.whl
  Using cached fastapi-0.115.9-py3-none-any.whl.metadata (27 kB)
  Using cached posthog-3.23.0-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached onnxruntime-1.21.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached opentelemetry_api-1.31.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.31.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.52b1-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_sdk-1.31.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached grpcio-1.71.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached bcrypt-4.3.0-cp39-abi3-win_amd6

In [5]:
!pip install sentence_transformers

In [6]:
!pip install gradio

In [8]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("data", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()

Intializing Chatbot.........


C:\Users\agile\AppData\Local\Temp\ipykernel_2916\2960288773.py:57: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
C:\Users\agile\AppData\Local\Temp\ipykernel_2916\2960288773.py:65: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Chatbot: Hello. It's nice to meet you. I'm here to listen and support you in a safe and non-judgmental space. How are you feeling today? Is there something on your mind that you'd like to talk about, or would you just like some company? I'm here to listen and help in any way I can.
Chatbot: Take Care of yourself, Goodbye!


In [9]:
create_vector_db()

<ipython-input-8-b60ab9fe9bb1>:22: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ChromaDB created and data saved


<ipython-input-8-b60ab9fe9bb1>:24: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


In [10]:
!pip install gradio

In [10]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import gradio as gr

def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("data", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


print("Intializing Chatbot.........")
llm = initialize_llm()

db_path = "chroma_db"

if not os.path.exists(db_path):
  vector_db  = create_vector_db()
else:
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(user_input, history=[]):
    if not user_input.strip():
        return "Please provide a valid input", history

    response = qa_chain.run(user_input)

    # Update history using OpenAI-style messages
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": response})

    return "", history


with gr.Blocks(theme = 'Respair/Shiki@1.2.1') as app:
    chatbot = gr.ChatInterface(fn=chatbot_response, title="Mental Health Chatbot")

app.launch(share=True)


Intializing Chatbot.........


c:\Users\agile\anaconda3\Lib\site-packages\gradio\components\chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/04/04 19:00:16 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


Traceback (most recent call last):
  File "c:\Users\agile\anaconda3\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\agile\anaconda3\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\agile\anaconda3\Lib\site-packages\gradio\blocks.py", line 2042, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\agile\anaconda3\Lib\site-packages\gradio\blocks.py", line 1587, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\agile\anaconda3\Lib\site-packages\gradio\utils.py", line 850, in async_wrapper
    response = await f(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\agile\a

In [12]:
# Import necessary libraries
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import gradio as gr

def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )
    return llm

def create_vector_db():
    # First make sure the data directory exists
    if not os.path.exists("data"):
        os.makedirs("data")
        print("Created /content/data directory. Please upload your PDF files there.")
        return None

    loader = DirectoryLoader("data", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()

    if not documents:
        print("No PDF files found in /content/data/ directory. Please upload some PDF files.")
        return None

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()

    print("ChromaDB created and data saved")

    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """
    You are a compassionate mental health chatbot. Respond thoughtfully to the following question:

    {context}

    User: {question}

    Chatbot:
    """
    PROMPT = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# Install required packages (for Colab)
try:
    import langchain_groq
except ImportError:
    print("Installing required packages...")
    !pip install langchain langchain-groq langchain-community gradio chromadb sentence-transformers

# Initialize the chatbot
print("Initializing Chatbot...")
llm = initialize_llm()

db_path = "chroma_db"

if not os.path.exists(db_path):
    vector_db = create_vector_db()
    if vector_db is None:
        print("Please upload PDF files to the /content/data/ directory and run this cell again.")
else:
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

# Only setup the QA chain if vector_db exists
if 'vector_db' in locals() and vector_db is not None:
    qa_chain = setup_qa_chain(vector_db, llm)

    def chatbot_response(user_input, chat_history):
        if not user_input.strip():
            return "Please provide a valid input."

        response = qa_chain.run(user_input)
        return response

    with gr.Blocks(theme='Respair/Shiki@1.2.1') as app:
        gr.Markdown("# Mental Health Chatbot")
        gr.Markdown("Ask any mental health related questions and get compassionate responses.")

        chatbot = gr.ChatInterface(
            fn=chatbot_response,
            title="Mental Health Chatbot",
            examples=["How can I manage anxiety?", "What are signs of depression?", "Techniques for stress relief"]
        )

    app.launch(share=True)
else:
    print("Vector database setup failed. Cannot launch chatbot.")

Initializing Chatbot...


c:\Users\agile\anaconda3\Lib\site-packages\gradio\components\chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/04/04 19:01:48 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


In [13]:
import os
import gradio as gr
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Initialize LLM
def initialize_llm():
    return ChatOpenAI(
        temperature=0,
        openai_api_key="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",  # Directly add key
        model_name="gpt-3.5-turbo"
    )

# Create ChromaDB from PDF documents
def create_vector_db(data_path="./data", db_path="./chroma_db"):
    loader = DirectoryLoader(data_path, glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory=db_path)
    vector_db.persist()
    print("ChromaDB created and data saved")
    return vector_db

# Setup Retrieval QA Chain
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """
    You are a compassionate mental health chatbot. Provide thoughtful responses.
    Context: {context}
    User: {question}
    Chatbot:
    """
    PROMPT = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT})

# Initialize components
llm = initialize_llm()
db_path = "./chroma_db"
if not os.path.exists(db_path) or not os.listdir(db_path):
    vector_db = create_vector_db()
else:
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
qa_chain = setup_qa_chain(vector_db, llm)

# Define chatbot response function
def chatbot_response(user_input, history=[]):
    if not user_input.strip():
        return "Please provide a valid input", history
    response = qa_chain.run(user_input)
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": response})
    return "", history

# Build Gradio Chatbot Interface
with gr.Blocks() as app:
    chatbot = gr.ChatInterface(fn=chatbot_response, title="Mental Health Chatbot", type="messages")
app.launch(share=True)


<ipython-input-13-23e4c9858327>:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  return ChatOpenAI(
<ipython-input-13-23e4c9858327>:50: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d3dc547dc947add644.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
print(os.getenv("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"))
os.environ["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"] = "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"


your_openai_api_key_here


In [14]:
pip install --upgrade langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.8 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.21
    Uninstalling langchain-0.3.21:
      Successfully uninstalled langchain-0.3.21
